# Export raw tick data using dedicated service - Multi identifiers - C#

### Overview
This sample demonstrates how to request from a dedicated raw data service **on-demand** tick by tick data `for a given instrument watchlist` and how to synchronize the different sources (trades and full order book). 

Raw tick by tick data is the primary data delivred by a specific data provider (exchange, third party, partner, in-house ...) and has not been processed or normalized yet.

### Inputs/outputs
Tick data extraction sample requires instrument's identifier, date time intervals as per inputs and exports tick by tick data file. 

### Services used
This sample uses *gRPC requests* in order to retrieve tick by tick dataset from the hosted service. The queried endpoint in this script are:
* *TickRawService*: to directly retrieve raw dataset from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

# Run tick by tick dataset extraction sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre

In [2]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using System.IO;
using System.Globalization;

using TickRawService = Systemathics.Apis.Services.Tick.V1.TickRawService;
using TickRawRequest = Systemathics.Apis.Services.Tick.V1.TickRawRequest;
using TickRawResponse = Systemathics.Apis.Services.Tick.V1.TickRawResponse;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
var headers = new Grpc.Core.Metadata();
try { headers = TokenHelpers.GetTokenAsMetaData(); } catch {}

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *raw tick data* service, we need to specify instrument identifier and a period of time. The following code snippets ensure required objects are created and proceed to request instantiation.

#### 3.1 Instrument selection

In [4]:
// Tha data is provided by ICE : let's use the ICE mapping codes to generate the identifier
// The ICE ticker
var ticker = "E:BNP";

#### 3.2 Time period delimitation

The following code setups time constraints

In [5]:
// Create time intervals
// Full order book data avaialble (sample) : from 2021-11-01 to 2021-11-12
var start = new DateTime(2021, 11, 01);
var end = new DateTime(2021, 11, 01);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
var timeInterval = new TimeInterval()
{
     StartTime = new TimeOfDay { Hours = 14, Minutes = 00, Seconds = 00 },
     EndTime = new TimeOfDay { Hours = 14, Minutes = 10, Seconds = 00 } 
};

The following code creates the market data request and service client:

In [6]:
// Generate constraints based on the previous space selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

// Generate raw tick data request
var request = new TickRawRequest() 
{ 
    Constraints = constraints
};

// We want to stream the ticks from those two Euronext sources (specific to provider "ICE")
// Full order book source
request.Identifiers.Add(new IdentifierAndLevel { 
    Provider = "ICE", 
    Ticker = ticker, 
    Exchange = "EQUITY_L2_973", 
    Level = Level.TradesAndBook });

// Trades only source
request.Identifiers.Add(new IdentifierAndLevel { 
    Provider = "ICE", 
    Ticker = ticker, 
    Exchange = "787", 
    Level = Level.Trades });

// Instantiate the raw tick data service
var service = new TickRawService.TickRawServiceClient(channel);

### Step 4: Retrieve tick by tick data

Tick by tick data is sent through a *stream*, the following code snippet exports tick by tick dataset in a file in the same folder as this sample:

In [7]:
// Get the tick raw data
var call = service.TickRaw(request, headers);

// Export tick raw data to a txt file
var ticksCount = 0;
var d = new DirectoryInfo("output"); if (!d.Exists) { d.Create(); }
using (var fs = new StreamWriter(File.OpenWrite($"output/{ticker}_raw_ticks_multi.csv")))
await foreach (var tickRawResponse in call.ResponseStream.ReadAllAsync())
{
    if (tickRawResponse.Raw != null)
    {
        var raw = tickRawResponse.Raw;
        ticksCount++; 
        var ss = Encoding.ASCII.GetString(raw.Payload.ToByteArray()).Substring(2);
        fs.WriteLine($"{raw.TimeStamp.ToDateTime():O},{raw.Mapping},{ss}");
        continue;
    }

    if (tickRawResponse.Mapping != null)
    {
        var mappings = tickRawResponse.Mapping;
        fs.WriteLine("TimeStamp,Mapping,RAW");
        foreach(var a in mappings.Values)
        {
            fs.WriteLine($",{a.Value} => {a.Identifier.Ticker} from {a.Identifier.Exchange},");
        }

        continue;
    }
}

The following code snippet is a summary with some metrics about the dateset export: 

In [8]:
Console.WriteLine($"Start date: {start:yyyy-MM-dd}");
Console.WriteLine($"Start date: {end:yyyy-MM-dd}");
Console.WriteLine($"Total ticks retrieved for {ticker}: {ticksCount:N0}");

Start date: 2021-11-01


Start date: 2021-11-01


Total ticks retrieved for E:BNP: 18,182
